# Projekt 3 - Sebesség mérés színekkel

Megszokhattuk már, hogy autó vezetés közben a gázra lépve a sebességmutató (vagy a fordulatszámmutató) elfordul, jelezve, hogy növekszik a sebességünk (a motor fordulatszáma). A modernebb (elektromos) autóknál mér nem analóg, hanem digitális kijelző van, ahol akár egy növő vastag csík jelzi a sebesség növekedését, akár változtatva a csík színét is a sebesség függvényében. 

Ebben a projektben egy RGB LED-et használunk majd a motor forgási sebességének megjelenítésére. A piros színtől (lassú) a kékig (gyors) folytonosan változtatjuk a LED színét. A sebesség változtatására nyomógombokat használunk. 

## Mit fogsz készíteni?

Egy 3-6V DC Motorból, 3 nyomógombból, 1 RGB LED-ből és egy L293D vezérlő (driver) chipből álló rendszert építünk, amit a Raspberry Pi-hoz kapcsolunk. A nyomógombok felelnek majd a motor ki/bekapcsolásáért, illetve a forgási sebességének megváltoztatásáért. A sebességet 0.1 egységenként növelhetjük vagy csökkenthetjük. Figyeljünk arra is, hogy a sebesség paraméterünk ne léphesse át a minimum 0 vagy a maximum 1 értékeket. A forgási sebesség nagyságát egy RGB LED különböző színeivel szimbolizáljuk. **Megjegyzés: Mindig használjunk megfelelő drivert a motorral összekötve, hogy csökkentsük bármelyik eszköz károsodásának lehetőségét.**

## Mit tanulsz meg?

A színes sebességmérő elkészítésével a következőket tanulod meg:

* Hogyan tudsz egy DC motort egy áramkörbe beiktatni a hozzá tartozó driverrel és nyomó gombokkal együtt.
* Hogyan programozd be a Raspberry Pi GPIO kimeneteleit a **gpiozero** modullal.
* **lambda** kifejezés használatát.
* Hogyan adjunk át a nyomógombnak olyan függvényeket, amiknek több bemeneti paramétere van.
* Hogyan programozzuk be a gomb különböző tulajdonságait egyszerre.
* Hogyan kezeljük a globális változókat.
* Hogy vezéreljük az RGB LED színét egy függvényen keresztül.

## A projekt részletekre bontása

* Elkészíteni az áramkört.
* Inicializálni a Motor, RGB LED és nyomógomb objektumokat.
* Definiálni egy függvényt, ami a motor ki- és bekapcsolásáért felel.
* Definiálni egy függvényt ami képes a bemenő paraméter függvényében színkódokat előállítani.
* Definiálni egy függvényt ami növeli vagy épp csökkenti a motor sebességét, a bemenő paraméterek függvényében. Egyben garantálja is, hogy az új sebesség ne legyen nagyobb (kisebb) mint a maximális (minimális) sebesség.  
* Hozzárendelni a megfelelő függvényeket a megfelelő gombokhoz.

## Áramköri elemek listája

a) [Raspberry PI](https://malnapc.hu/yis/raspberry-pi/rpi-panelek) 

b) DC Motor: [itt vásárolhatsz](https://www.tme.eu/hu/details/oky5022-1/dc-motorok/okystar/)

c) [Jumper wires female/male](https://www.ret.hu/shop/product/e-call/jumper-vezetek-szet_53-22-63) 

d) L293D vezérlő: [itt vásárolhatsz](https://www.tme.eu/hu/details/l293d/motor-es-pwm-driverek/stmicroelectronics/)

e) 3 db nyomó gomb: [itt vásárolhatsz](https://hu.farnell.com/schurter/1301-9320/switch-smd-push-12-5mm/dp/1217772?gclid=Cj0KCQjwzZj2BRDVARIsABs3l9K-ACTnuRr-dLVcDUKleNfECM3H3kWS_RfWtmXGMXVJeY9otda4dDcaAvGLEALw_wcB&gross_price=true&mckv=sSFnjERxR_dc|pcrid|434487710397|plid||kword||match||slid||product|1217772|pgrid|101346804139|ptaid|pla-389347076066|&CMP=KNC-GHU-SHOPPING-switches-relays-NEWSTRUCTURE-13-MAY-20) vagy [építhetsz]()

f) 4 db AAA vagy AA [ceruza elem]()

g) RGB LED, $I_{max}$ = 20 mA-es: [itt vásárolhatsz](https://www.tme.eu/hu/details/osta56a1a-c/tht-led-diodak-egyeb/optosupply/)

## A kapcsolási rajz

<img src="prog02_schema.png" width=600 height=400 />

A fenti ábrához hasonlóan kapcsoljuk össze az áramköri elemeket és a Raspberry Pi-t. Az áramkör részletes összekötési magyarázata a [Motorok]() című bevezető leírásban található.

1) Az L293D driver földelés lábait kössük össze egymással és az átellenes oldallal (fekete drót). Egyben az áramforrás (4 db AAA elem a rajzon) negatív végét és a Raspberry Pi egyik földelését is kössük össze az L293D driver földelésével. Így minden elem azonos földelésen lesz.

2) Az áramforrás pozitív végét kapcsoljuk a driver *VS* lábára (*8*-as láb, piros drót). Ez adja a feszültséget a motor meghajtásához. 

3) Kapcsoljuk a Raspberry Pi +5V-os tüskéjét a driver *VSS* (*16*-os) lábával össze (fehér drót). Ez adja a driver működéséhez nélkülözhetetlen tápot. 

4) A Raspberry Pi *GPIO25*-ös tüskéjét kapcsoljuk az *EN1* (*1*-es) lábra (okkersárga drót). A driver ezen lába szolgál arra, hogy aktiválja a driver *IN1 (2), IN2 (7), OUT1 (3) és OUT2 (6)* lábait, amennyiben magas állapotba kerül. Alacsony állapotból az előbb említett lábak nem kapnak áramot. 

5) A Raspberry Pi *GPIO23*-as és *GPIO24*-es tüskéit kapcsoljuk az *IN1 (2), IN2 (7)* lábakra a driveren (szürke és narancssárga drótok). Felváltva aktiválva őket vagyunk képesek előre és hátrafele forgatni a motort attől függően, hogy épp melyik van magas állapotban. 

6) A motor két kimenetét (citromsárga és zöld drótok) kapcsoljuk a driver *OUT1 (3) és OUT2 (6)* lábaira. Az *IN1 és IN2* lábak állapota szabályozza, hogy ezek a lábak, azaz maga a motor, kap-e feszültséget.

7) Illesszünk be egy nyomógombot a breadboardba és annak egyik lábát kapcsoljuk a földelésre a másikat pedig a *GPIO17*-es tüskére (lila drót).

8) Illesszünk be egy másik nyomógombot a breadboardba és annak egyik lábát kapcsoljuk a földelésre a másikat pedig a *GPIO27*-es tüskére (barna drót).

9) Illesszünk be egy harmadik nyomógombot is a breadboardba és annak egyik lábát kapcsoljuk a földelésre a másikat pedig a *GPIO22*-es tskére (pink drót).

10) Az RGB LED leghosszabb lábát (közös katód esetén) kössük össze a földeléssel egy jumper drót segítségével.

11) A többi láb sorába az ábrának megfelelően kössük be az ellenállásokat.

12) A piros (R) lábhoz tartozó ellenállás sorát kössük a **9**-es GPIO tüskéhez, a zöldet (G) a **10**-eshez és a kéket a **11**-hez.

**Kétszer is ellenőrizzük le, hogy a bekötésünk rendben van-e. A félrekötött tüskék nagyban növelik a motor vagy a Raspberry Pi tökremenési esélyeit. A motort SEMMIKÉPP SE tápláljuk és irányítsük direktben a Raspberry Pi-ről, mert az szinte biztosan a számítógép sérükéséhez vezet.**

## A kód
Nyissunk meg egy új python fájlt és mentsük el pl. ```dcmotor_speed.py``` név alatt.

### A DC motor kipróbálása

Miután elkészítettük az áramkört, meg kell írnunk a kódot ami utasítja a Raspberry Pi-t, a motor mozgatására.

Először betöltjük a ```gpiozero``` modulból a ```Motor``` objektumot ami lehetővé teszik, hogy a Raspi kommunikáljon a motorral a hozzá tartozó vezérlővel a **GPIO** tüskéken keresztül. Emellett beimportáljuk a ```sleep``` függvényt is, hogy szünetet tarthassunk a python kód lefutásában. 

```dcmotor_speed.py```:

In [ ]:
from gpiozero import Motor
from time import sleep

Az objektum beolvasása után inicializálunk egy ```Motor``` objektumot, amit ```motor```-nak nevezünk el. Egyben megmondjuk az objektumnak, hogy a fizikai motor előre forgatásért felelő drótját a *23*-as számú **GPIO** tüskére csatlakoztattuk, míg a hátrafelé forgatásért felelő drótot a *24*-es számú **GPIO** tüskére. Emellett a driveren a kimeneteket aktiváló drótot a *25*-ös számú **GPIO** tüskére csatlakoztattuk. 

```dcmotor_speed.py```:

In [ ]:
from gpiozero import Motor
from time import sleep

motor = Motor(forward=23, backward = 24, enable=25)

Ezután készen állunk arra, hogy leteszteljük a motor mozgatását és az objektumhoz tartozó egyéb metódusok működését. Elsőnek a motort előre forgatjuk ```motor.forward()```, 5 másodpercen keresztül, majd megállítjuk a ```motor.stop()``` paranccsal és leteszteljük, hogy hátrafele is tud-e forogni, ```motor.backward()```. Hagyjuk forogni 5 másodpercet, majd újra leállítjuk. 

```dcmotor_speed.py```:

In [ ]:
from gpiozero import Motor
from time import sleep

motor = Motor(forward=23, backward = 24, enable=25)

motor.forward()
sleep(5)
motor.stop()

motor.backward()
sleep(5)
motor.stop()

Ha lefuttatjuk ezt a kódot, kiderül, hogy mindent rendben kötöttünk-e össze az áramkörben. Ha nem az elvárt mozgásokat tapasztaljuk, akkor meg kell keresni a hibát. Nagy valószínűséggel az áramköri kapcsolásokkal van gond.

### A gombok tesztelése

Következő lépésben beimportáljuk a gombokért felelős ```Button``` objektumot is és definiálunk két gombot. Az egyiket a *GPIO17*-re a másikat pedig a *GPIO27*-re kötjük. 

```dcmotor_speed.py```:

In [ ]:
from gpiozero import Motor, Button
from time import sleep

motor = Motor(forward=23, backward = 24, enable=25)
button_start_stop = Button(17)
button_faster = Button(27)
button_slower = Button(22)

Ahhoz, hogy leteszteljük helyesen kötöttük-e be a gombokat és, hogy azok működnek, írunk egy segédfüggvényt ```test()```, amit a gomb lenyomásakor meghívunk majd. A függvény meghívásakor egyszerűen csak kiíratjuk a képernyőre, hogy a gombot megnyomtuk. Végül ezt a függvényt hozzá kell rendelnünk a gombhoz, méghozzá most a gomb lenyomásának pillanatához a ```when_pressed``` metódushoz.

```dcmotor_speed.py```:

In [ ]:
from gpiozero import Motor, Button
from time import sleep

motor = Motor(forward=23, backward = 24, enable=25)
button_start_stop = Button(17)
button_faster = Button(27)
button_slower = Button(22)

def test():
    print('Button is pressed')
    
button_start_stop.when_pressed = test
button_faster.when_pressed = test
button_slower.when_pressed = test

while True:
    pass

A kód végén még elindítunk egy végtelen ciklust, ami nem csinál semmit, így csak a gombok lenyomása lesz az egyetlen egy esemény amit a számítógép jelezhet. 

Mentsük le a kódot és futtasuk le a pythonnal. Teszteljük, hogy a gombok lenyomására megjelenik-e az üzenet a képernyőn. 

### Az RGB LED tesztelése

Először betöltjük a gpiozero modulból az RGBLED objektumot ami lehetővé teszik, hogy a Raspi kommunikáljon a LED-del a GPIO tüskéken keresztül. Ez mellett beolvassuk még a sleep függvényt is, amit a python várakoztatására használunk majd.

Ezután leteszteljük, hogy mind a három színünk jól működik-e. Egyenként bekapcsoljuk őket, majd 1 másodpercet várva kikapcsoljuk és áttérünk a következő színre.

In [ ]:
from gpiozero import Motor, Button, RGBLED
from time import sleep

motor = Motor(forward=23, backward = 24, enable=25)  # Motor(23,24,25)
led = RGBLED(red=9, green=10, blue=11, active_high = True)
button_start_stop = Button(17)
button_faster = Button(27)
button_slower = Button(22)

led.red = 1  # full red
sleep(1)
led.red = 0
led.green = 1
sleep(1)
led.green = 0
led.blue = 1
sleep(1)
led.blue = 0

Ha minden rendben működött, akkor a program lefuttatásakor mind a három szín megjelenik egyenként a diódán.

### Színt változtató függvény definiálása

Röviden a ```lambda``` kifejezés az egy egysoros függvény definiálás, ami nem él az egész programon keresztül, csak abban az egy sorban. Főként akkor alkalmazzuk, ha egy egyszer használatos egysoros függvényt akarunk definiálni. A ```lambda``` kifejezés szabálya, hogy a ```lambda``` parancsszavat követi(k) a bemeneti paraméter(ek), majd kettőspont és utána a függvény amit definiálni akartunk. Nincs megszabva, hány bemeneti paramétert adunk meg. Az alábbi példában egy névtelen függvényt definiálunk, ami a bemenő paraméterhez (x) 3-at ad hozzá.

In [3]:
(lambda x: x + 3)(2)

5

Hogy lefusson, a lambda kifejezést zárójelbe tettük és utána egy másik zárójelben megadtuk a bemeneti paraméter értékét. Hogy kicsit hasonlóbb legyen a szokásos ```def``` módszerhez, a ```lambda``` kifejezést hozzárendelhetjük egy változóhoz, azaz egy függvénynévhez, amit később hívunk meg. Ezt ritkán szokták alkalmazni, de így nézne ki:

In [6]:
func = lambda x,y: x*y
func(2,3)

6

A fenti példában a két bemenő paraméterrel rendelkező ```lambda``` kifejezést, ami a két paramétert összeszorozza, hozzárendeltük a ```func``` nevű változóhoz. Egy sorral később hívtuk csak meg a függvényt az adott bemenő paraméterekkel. 

A ```lambda``` kifejezést úgy is használhatjuk, hogy nem definiálunk bemenő paramétert, így meghíváskor csak üres zárójelet írunk. Ebben az esetben, a függvény tartalma csak végrehajtódik. 

In [7]:
(lambda : print('hello raspberry'))()

hello raspberry


Mi az alábbiakban úgy fogjuk használni a ```lambda``` kifejezést, hogy nem adunk neki bemenő paramétert, viszont a testében egy már előre definiált másik függvényt fogunk meghívni, aminek viszont lesz bemenő paramétere. Egy példa erre:

In [9]:
def func(text):
    print(text)

(lambda : func('I like hamsters'))()

I like hamsters


In [10]:
(lambda : func('Hamsters like carrot'))()

Hamsters like carrot


Fent definiáltunk egy ```func(text)``` függvényt, aminek a bemenő paramétere egy string, a funkciója pedig, hogy kinyomtassa a megadott szöveget. A ```lambda``` kifejezésben ezt a függvényt hívtuk meg különböző szövegekkel, így mindig más és más nyomtatott szöveget kaptunk. 

Első ránézésre ez egy igen kitekert módja annak, hogy elérjük a célunk, hiszen elég lett volna kétszer meghívni a függvényünket, de látjuk majd, hogy a gombokhoz való függvényrendelésnél ez a módszer kifejezetten hasznos tud lenni.

### Függvény a motor be- és kikapcsolására

Most, hogy megismerkedtünk a ```lambda``` kifejezéssel, használjuk azt, hogy a különböző gombok lenyomásával különböző üzenetet tudjunk megjeleníteni. Kezdetnek írjük át a ```test``` függvényünket, hogy fogadjon bemenő paramétert:

In [13]:
def test(txt):
    print(f'{txt} button was pressed!')

Rendeljük ezt a függvényt a gombok ```when_held``` metódusához, de különböző szöveggel az egyik gomb és a másik gomb esetében. Végrehajtáskor ez a függvény addig nyomtatja majd ki az üzenetünket, amíg a gombot lenyomva tartjuk.

```dcmotor_speed.py```:

In [1]:
from gpiozero import Motor, Button

motor = Motor(forward=23, backward = 24, enable=25)  # Motor(23,24,25)
button_forward = Button(17)
button_backward = Button(27)

button_forward.hold_time = 0.2
button_forward.hold_repeat = True

button_backward.hold_time = 0.2
button_backward.hold_repeat = True

def test(txt):
    print(f'{txt} button was pressed!')

button_forward.when_held = lambda : test('Forward')

button_backward.when_held = lambda : test('Backward')

while True:
    pass

Lefuttatva ezt a kódot, azt kellene tapasztalnunk, hogy ha az előre forgató gombot nyomjuk le, akkor a 'Forward Button was pressed' szöveg nyomtatódik ki folytonosan a képernyőre amíg a gombot nyomva tartjuk. Ha a hátrafele forgató gombot nyomjuk meg, akkor pedig a 'Backward button was pressed' szöveg jelenik meg folytonosan. 

A ```lambda``` kifejezésre itt azért volt szükség mert, a ```when_held``` metódus egy függvényt, és nem egy meghívott függvényt kér. A ```lambda``` kifejezés pont egy függvényt definiál, aminek itt nincs bemenő paramétere, de a testében egy másik előre definiált függvényt hívunk meg különböző bemeneti paraméterekkel, amik végre is hajtódnak a gomb lenyomásával. 

### A motor sebességét vezérlő függvény definiálása

Ebben a bekezdésben a ```test``` függvényt lecseréljük a valós függvényre, ami felelni fog a motor megmozdításáért. Ezt ```rotate(button, m, direction='forward')```-nak fogjuk hívni, és látható, hogy több bemenő paraméterrel rendelkezik. Az első a lenyomott gombot szimbolizálja, a második azt a motort amit mozgatni szeretnénk, a harmadik pedig megmondja milyen irányba forgassuk a motort. 

A függvény testében először, a biztonság kedvéért, leellenőrizzük, hogy a paraméterként megadott gomb (feltehetőleg, az amit lenyomtunk), valóban le van-e nyomva, ```button.is_pressed```. Ha igen, akkor egy ```if``` szerkezetben leellenőrizzük, hogy a forgási irány az előre vagy hátra. Ha ```direction == 'forward'```, azaz az irány az előre, akkor kinyomtatjuk a képernyőre, hogy mozgás előre, illetve bakpcsoljuk a motor forgását a ```m.forward()``` függvénnyel. Ha ```direction == 'backward'```, akkor kinyomtatjuk a képernyőre, hogy mozgás hátrafele történik és bakpcsoljuk a motor forgását a ```m.backward()``` függvénnyel. 

In [ ]:
def rotate(button, m, direction='forward'):
    if button.is_pressed:
        if direction == 'forward':
            print('Moving forward')
            return m.forward()
        if direction == 'backward':
            print('Moving backward')
            return m.backward()       

Ha mindent jól gépeltünk le, akkor a motor, ennek a függvénynek a segítségével képes lesz forogni. Viszont, hiába engednénk fel a gombot, a motor továbbra is forogna a kijelölt irányba, mert nem adtunk neki stop parancsot. Hogy ezt a csorbát orvosoljuk definiálunk egy ```motor_stop(button, m)``` függvényt, ami megállítja a bemenő paraméterként megadott motort. 

In [ ]:
def motor_stop(button, m):
    m.stop()

Ebben a projektben nem lenne szükség külön függvény definiálására a motor megállításához, mivel csak egy motorunk van, de ha több lenne, akkor ugyanazt a függvényt meghívva, csak más bemenő paramétert adva, képesek lennénk az összes motor leállítására. 

### A függvények gombokhoz való rendelése

Utolsó lépésként az előbb definiált függvényeket beillesszük a kódunkba és hozzárendeljük őket a megfelelő gombok megfelelő metódusához, használva a ```lambda``` kifejezést. A ```when_held``` metódushoz a ```rotate``` függvényt rendeljük, míg a ```when_released``` metódushoz a ```motor_stop``` függvényt, a megfelelő bemeneti paraméterekkel. Az utóbbi metódus akkor fogja lefuttatni a hozzárendelt függvényt, amikor felengedtük a gombot. Evvel biztosítjuk azt, hogy a motor forog mindaddig, amíg a gombot lenyomva tartjuk, de azonnal megáll ha felengedjük a gombot.  

```dcmotor_speed.py```:

In [ ]:
from gpiozero import Motor, Button, RGBLED

motor = Motor(forward=23, backward = 24, enable=25)  
led = RGBLED(red=9, green=10, blue=11, active_high = True)
button_start_stop = Button(17)
button_faster = Button(27)
button_slower = Button(22)

on_off = False  


def start_stop(button, m):
    global on_off, led
    if on_off:
        on_off = False
        m.stop()
        led.off()
        print('Motor off...')
    else:
        on_off = True
        m.forward()
        color = value2rgb(0, 1, m.value)
        led.color = color
        print('Motor on...')

def speed(button, increment = 0.1):
    global led, motor
    new_speed = motor.value + increment
    if new_speed > 0 and new_speed < 1.01:
        print(f'New speed: {new_speed}')
        motor.value = new_speed
        color = value2rgb(0, 1, new_speed)
        led.color = color
    else:
        print('Motor speed is out of range of 0 and 1.')
        print(f'Current speed is: {motor.value}')

def value2rgb(minimum, maximum, value):
    minimum, maximum = float(minimum), float(maximum)
    ratio = 2 * (value-minimum) / (maximum - minimum)
    b = int(max(0, 255*(1 - ratio)))/255
    r = int(max(0, 255*(ratio - 1)))/255
    g = (255 - b - r)/255
    return r, g, b


button_start_stop.when_pressed = lambda : start_stop(button_start_stop, motor)
button_faster.when_pressed = lambda : speed(button_faster, increment=0.1)
button_slower.when_pressed = lambda : speed(button_slower, increment=-0.1)

while True:
    pass

## A projekt tesztelése

Miután összeszereltük az áramkört és a kódot is megírtuk, amit pl. ```dcmotor_speed.py``` név alatt mentettünk el, megnyithatunk a Raspberry Pi operációs rendszerén egy terminált. A terminálban a ```cd 'mappa név'``` paranccsal elnavigálunk abba a mappába, ahova a ```dcmotor_speed.py```-t elmentettük. Ott begépelve a ```python dcmotor_speed.py``` parancsot, letesztelhetjük a programunk működését. Ha minden jól megy akkor a start gomb megnyomása után a LED bekapcsol és a motor elkezd forogni. Ha változtatjuk a sebességét, akkor a LED is változtatja a színét. Ezenfelül a program nem engedi, hogy a sebesség a 0 és 1 intervallumon kívüli értéket vegyen fel.

Hibaüzenetek esetén ki kell deríteni mi lehetett a probléma, pl. elgépelés, egy modul hiányzik, sorok megfelelő behúzása, idézőjel lemaradása stb. A hibaüzenet legtöbbször segít abban, hogy melyik sorban találta a hibát és hogy mi volt az. Egy kis gyakorlással bele lehet jönni azok értelmezésébe, valamint interneten is rá lehet keresni a hibaüzenet jelentésére és annak lehetséges elhárítására.

## Mit lehet javítani/továbbfejleszteni?

* LED-del jelezzük, ha a motor aktív.
* RGB LED zölden jelezze ha előre és pirosan ha hátrafele forog a motor.

Írd meg kommentben, hogy szerinted mivel lehetne még feldobni ezt a kis programot!

## Referencia

1) gpiozero leírás - [https://gpiozero.readthedocs.io/en/stable/recipes.html](https://gpiozero.readthedocs.io/en/stable/recipes.html)

2) Motor objektum leírása - [https://gpiozero.readthedocs.io/en/stable/api_output.html#motor](https://gpiozero.readthedocs.io/en/stable/api_output.html#motor)

3) L293D driver leírása - [https://www.ti.com/lit/ds/symlink/l293.pdf](https://www.ti.com/lit/ds/symlink/l293.pdf)

4) Button objektum leírása - [https://gpiozero.readthedocs.io/en/stable/api_input.html?highlight=button#button](https://gpiozero.readthedocs.io/en/stable/api_input.html?highlight=button#button)

5) lambda kifejezés - [https://www.programiz.com/python-programming/anonymous-function](https://www.programiz.com/python-programming/anonymous-function)

6) RGBLED objektum leírása - [https://gpiozero.readthedocs.io/en/stable/api_output.html#rgbled](https://gpiozero.readthedocs.io/en/stable/api_output.html#rgbled)